In [1]:
import pandas as pd
import numpy as np
import re

df=pd.read_csv('glassdoor_data.csv')
df=df[df['Salary Estimate']!='-1']

df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'Employer est.' in x.lower() else 0)
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0].replace('K','').replace('$','').replace('Per Hour',''))
print(salary[13:20])
# salary=minusk.apply(lambda x: x.replace('Per Hour','').replace('Employer est.',''))

# Extract minimum and maximum salary from "Salary Estimate" column
def extract_min_salary(salary):
    try:
        split_values = salary.split('-')
        if len(split_values) >= 2:
            return float(split_values[0])
    except:
        pass
    return None

def extract_max_salary(salary):
    try:
        split_values = salary.split('-')
        if len(split_values) >= 2:
            return float(split_values[1])
    except:
        pass
    return None

df['Min Salary'] = salary.apply(extract_min_salary)
df['Max Salary'] = salary.apply(extract_max_salary)

df['avg_salary']=(df['Min Salary']+df['Max Salary'])/2

df['Rating'] = df['Rating'].apply(lambda x: x.split(' ')[0])
# df['Company Name'] = df['Company Name'].apply(lambda x: x.split(' ')[0])

def remove_numerical_part(value):
    value = re.sub(r'\d+', '', value)  # Remove numerical part
    value = re.sub(r'[^\w\s]', '', value)  # Remove special characters
    return value.strip()  # Remove leading and trailing whitespaces


df['Company Name'] = df['Company Name'].apply(remove_numerical_part)

def extract_jobstate(location):
    splitval = location.split(',')
    if len(splitval)>=2:
        return (splitval[1])

df['job_state'] = df['Location'].apply(extract_jobstate)

# df.job_state.value_counts()

df['age']=df['Founded'].apply(lambda x: x if x<1 else (2023-int(x)))

df['python']=df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

df['excel']=df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

df['sql']=df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)

df.sql.value_counts()

def extract_company(ownership):
    splitval=ownership.split('-')
    if len(splitval)>1:
        return splitval[1]
df['Type of ownership']=df['Type of ownership'].apply(extract_company)

ownership_dummies = pd.get_dummies(df['Type of ownership'])
df = pd.concat([df, ownership_dummies], axis=1)
df.rename(columns={' Private ': 'Private'}, inplace=True)
df.rename(columns={' Public ': 'Public'}, inplace=True)

column_name1 = ' Private'
column_name2 = ' Public'
df.rename(columns={column_name1: column_name1.strip()}, inplace=True)
df.rename(columns={column_name2: column_name2.strip()}, inplace=True)

df['desc_len']=df['Job Description'].apply(lambda x: len(x))

# df.columns
df_out = df.drop(['Unnamed: 0','Headquarters','Competitors','1'], axis =1)

df_out.to_csv('data_cleaned.csv',index=False)

13    50.00 - 55.00  
14           51 - 75 
15           56 - 84 
16           48 - 72 
17    38.00 - 40.00  
18           44 - 68 
19           54 - 58 
Name: Salary Estimate, dtype: object


In [2]:
df_out

Job Title                Salary Estimate  \
0             Data Analyst I   $41K - $55K (Glassdoor est.)   
1     Reporting Data Analyst   $64K - $92K (Glassdoor est.)   
2        Junior Data Analyst   $45K - $69K (Glassdoor est.)   
3          Data Analyst(SQL)  $64K - $106K (Glassdoor est.)   
4         Data Analyst (Jr.)   $80K - $100K (Employer est.)   
..                       ...                            ...   
195             Data Analyst   $49K - $70K (Glassdoor est.)   
196      Junior Data Analyst    $59K - $95K (Employer est.)   
197  Junior Business Analyst    $65K - $90K (Employer est.)   
198             Data Analyst  $67K - $100K (Glassdoor est.)   
199             Data Analyst   $51K - $75K (Glassdoor est.)   

                                       Job Description Rating  \
0    Job Summary:\nIf you live in or around the San...    4.3   
1    Medlogix, LLC delivers innovative medical clai...    2.6   
2    Hintz and Co is a boutique full service media ...    3.8   
3    Job Title: Data Analyst (SQL)\nJob Summary: We...    4.1   
4    Department of Veterans Affairs experience? Do ...    4.0   
..                                                 ...    ...   
195  Chicago Fire Football Club is a Major League S...    3.8   
196  Junior Data Analyst\nThis role is open to our ...    4.1   
197  Who We Are: ThinkTek is a fast-growing Certifi...    5.0   
198  Aura is on a mission to create a safer interne...    4.3   
199  Data Analyst:\nAt Hamilton Beach Brands, Inc.,...    3.3   

                      Company Name           Location                  Size  \
0           Firstmark Credit Union    San Antonio, TX  201 to 500 Employees   
1                     MEDLOGIX LLC       Lansdale, PA               Unknown   
2                     Hintz and Co    Los Angeles, CA     1 to 50 Employees   
3                Leading Torch LLC          Plano, TX     1 to 50 Employees   
4    Ben Allegretti Consulting BAC     Washington, DC               Unknown   
..                             ...                ...                   ...   
195                Chicago Fire FC     Bridgeview, IL   51 to 200 Employees   
196                 Hotwire Global  San Francisco, CA  201 to 500 Employees   
197                   ThinkTek LLC        Fairfax, VA     1 to 50 Employees   
198                           Aura         Boston, MA  201 to 500 Employees   
199      Hamilton Beach Brands Inc     Glen Allen, VA  201 to 500 Employees   

     Founded Type of ownership                              Industry  ...  \
0       1932           Private                     Banking & Lending  ...   
1         -1           Private                    Insurance Carriers  ...   
2       1979           Private  Architectural & Engineering Services  ...   
3         -1           Private                                    -1  ...   
4       2007              None                     National Agencies  ...   
..       ...               ...                                   ...  ...   
195       -1           Private                   Sports & Recreation  ...   
196     2000            Public        Advertising & Public Relations  ...   
197       -1           Private                                    -1  ...   
198     2017           Private                  Software Development  ...   
199     1904            Public        Consumer Product Manufacturing  ...   

    Max Salary avg_salary  job_state  age  python  excel  sql Private  Public  \
0         55.0       48.0         TX   91       0      0    0       1       0   
1         92.0       78.0         PA   -1       0      0    0       1       0   
2         69.0       57.0         CA   44       0      0    0       1       0   
3        106.0       85.0         TX   -1       0      0    1       1       0   
4        100.0       90.0         DC   16       0      0    0       0       0   
..         ...        ...        ...  ...     ...    ...  ...     ...     ...   
195       70.0       59.5         IL   -

In [3]:
df_out[13:20]

Job Title                           Salary Estimate  \
13        Data Reporting Analyst  $50.00 - $55.00 Per Hour (Employer est.)   
14                  Data Analyst              $51K - $75K (Glassdoor est.)   
15                  Data Analyst              $56K - $84K (Glassdoor est.)   
16                  Data Analyst              $48K - $72K (Glassdoor est.)   
17                  Data Analyst  $38.00 - $40.00 Per Hour (Employer est.)   
18        eCommerce Data Analyst              $44K - $68K (Glassdoor est.)   
19  Business Analyst - Mid level               $54K - $58K (Employer est.)   

                                      Job Description Rating  \
13  The primary purpose of this position is to pro...    3.6   
14  Data Analyst:\nAt Hamilton Beach Brands, Inc.,...    3.3   
15  Do you want to work for a Championship organiz...    4.5   
16  Morton Buildings, Inc. is the industry leader ...    3.9   
17  Data Analyst\nThis is a Hybrid Role, minimum 2...    5.0   
18  Position Summary:\nThe eCommerce Data Analyst ...    3.8   
19  We are seeking a mid-level Business Analyst wh...    4.7   

                           Company Name             Location  \
13                    Devcare Solutions               Remote   
14            Hamilton Beach Brands Inc       Glen Allen, VA   
15                           Miami HEAT            Miami, FL   
16                 Morton Buildings Inc           Morton, IL   
17                              Codetru  North Brunswick, NJ   
18  American Freight Management Company         Delaware, OH   
19                             Accel BI               Remote   

                      Size  Founded Type of ownership  \
13     51 to 200 Employees     2005           Private   
14    201 to 500 Employees     1904            Public   
15  1001 to 5000 Employees     1987           Private   
16  1001 to 5000 Employees     1903           Private   
17     51 to 200 Employees     2016           Private   
18        10000+ Employees     1977           Private   
19       1 to 50 Employees     2000           Private   

                                   Industry  ... Max Salary avg_salary  \
13  Information Technology Support Services  ...       55.0       52.5   
14           Consumer Product Manufacturing  ...       75.0       63.0   
15                      Sports & Recreation  ...       84.0       70.0   
16                             Construction  ...       72.0       60.0   
17  Information Technology Support Services  ...       40.0       39.0   
18       Home Furniture & Housewares Stores  ...       68.0       56.0   
19  Information Technology Support Services  ...       58.0       56.0   

    job_state  age  python  excel  sql Private  Public  desc_len  
13       None   18       0      0    0       1       0      1171  
14         VA  119       0      0    0       0       1      1248  
15         FL   36       0      0    1       1       0       800  
16         IL  120       0      0    0       1       0       642  
17         NJ    7       0      0    0       1       0       857  
18         OH   46       0      0    0       1       0       765  
19       None   23       0      0    0       1       0       695  

[7 rows x 25 columns]